In [6]:
-- require 'loadcaffe';
-- require 'image';
-- require 'cunn';
-- require 'cudnn';
require 'nngraph';

function createFullModel(B_model, C_model)
	local image_feat1 = nn.Identity()();
	local image_feat2 = nn.Identity()();
	local image_feat3 = nn.Identity()();
	local question = nn.Identity()();

	local confidence = B_model({question, image_feat3});
	local scores = C_model({image_feat1, image_feat2, question, confidence});
	
	nngraph.annotateNodes();
	return nn.gModule({image_feat1, image_feat2, image_feat3, question}, {scores});
end

B_model = torch.load('B_model_nn.t7')
C_model = torch.load('C_model.t7')

-- B_model:cuda()
B_model:evaluate()

-- C_model:cuda()
C_model:evaluate()

BC_model = createFullModel(B_model, C_model)
-- ABC_model = createFullModel(A_model, B_model, C_model, encoders);

-- put all models on cuda and in evalaute mode

-- BC_model:cuda()
BC_model:double()
-- BC_model:evaluate()

-- Use a typical generic gradient update function
function trainStep(model, input, target, criterion, learningRate)
    local pred = model:forward(input)
    local err = criterion:forward(pred, target)
    local gradCriterion = criterion:backward(pred, target)

    model:zeroGradParameters()
    model:backward(input, {gradCriterion[1], gradCriterion[2]})
    C_model:updateParameters(learningRate)

--     print('pred = ', pred[1][1], pred[2][1])
    print('err = ', err)
end

-- read preprocessed feature vectors
feat_vecs = torch.load('feat_vecs.t7')

-- generate random label
-- label = torch.bernoulli() + 1
label = 2
target = 2*label-3

-- generate dummy data
image_feat = {}
image_feat[1] = feat_vecs[10]
image_feat[2] = feat_vecs[20]
image_feat[3] = image_feat[label]:clone()

-- generate a dummy question
ques = torch.Tensor(42):fill(0)
ques[3] = 1

-- dummy forward propagation through the model
output = BC_model:forward({image_feat[1], image_feat[2], image_feat[3], ques})
print(output[1][1], output[2][1])

-0.19407528740524	0.079775995626213	


In [7]:
-- for i = 1, #BC_model.modules do
--     mod = BC_model.modules[i]
--     print(mod.train)
-- end

In [8]:
wb_old = B_model.modules[2].modules[3].weight:clone()
wc1_old = C_model.modules[8].modules[1].weight:clone()
wc2_old = C_model.modules[8].modules[3].weight:clone()

-- dummy training
crit = nn.MarginRankingCriterion(0.1)
lr = 1
print(target)
for i = 1, 10 do
    trainStep(BC_model, {image_feat[1], image_feat[2], image_feat[3], ques}, target, crit, lr)
end

1	


err = 	0.37385128303145	


err = 	0	


err = 	0	


err = 	0	


...

err = 	0	


err = 	0	


err = 	0	


err = 	0	


err = 	0	


In [4]:
wb = B_model.modules[2].modules[3].weight:clone()
wc1 = C_model.modules[8].modules[1].weight:clone()
wc2 = C_model.modules[8].modules[3].weight:clone()

In [5]:
wb_diff = wb_old - wb
wc1_diff = wc1_old - wc1
wc2_diff = wc2_old - wc2
print(torch.max(torch.abs(wb_diff)))
print(torch.max(torch.abs(wc1_diff)))
print(torch.max(torch.abs(wc2_diff)))

0.017086426887453	


0	
0	
